# Imputer:

- substitui valores ausentes
- deve se escolher estratégia de substituição (média, mediana ou moda)
- pode substituir qualquer outro valor, por exemplo: "0" > utilizar parâmetro 
setMissingValue = <valor a ser substituído)
- só funciona para valores numéricos

In [5]:
#imports

import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init() # inicializa interface pyspark jupyter
spark = SparkSession.builder.appName("imputer").getOrCreate()
#inicializa sessão no spark com nome importação

In [6]:
from pyspark.ml.feature import Imputer

In [7]:
carros = spark.read.csv("CarrosNAN.csv", header=True, inferSchema=True, sep=";")

In [8]:
#Identificação de nulos
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|       null|             39|2875| null|        0|          1|      4|          4|110|
|    228|        0|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        0|       null|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        0|        360|            315|null| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| null|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [11]:
#Estratégia default = média (para substituição)
imput = Imputer(inputCols=["Cilindradas", "Peso"], outputCols=["novaCilindradas", "novaPeso"])
modelo = imput.fit(carros)
carros = modelo.transform(carros)
carros.select("novaCilindradas", "novaPeso").show()

+---------------+--------+
|novaCilindradas|novaPeso|
+---------------+--------+
|            160|     262|
|            848|    2875|
|            108|     232|
|            848|    3215|
|            360|    1318|
|            225|     346|
|            360|     357|
|           1467|     319|
|           1408|     315|
|           1676|     344|
|           1676|    1318|
|           2758|     407|
|           2758|     373|
|           2758|     378|
|            472|     525|
|            848|    5424|
|            440|    5345|
|            787|      22|
|            757|    1615|
|            711|    1835|
+---------------+--------+
only showing top 20 rows



In [14]:
# Estratégia de substituir pela mediana: (0>mediana(6))
imput = Imputer(inputCols=["Cilindros"], outputCols=["novaCilindros"])
imput.setStrategy("median").setMissingValue(0).fit(carros).transform(carros).select("Cilindros","novaCilindros").show()

+---------+-------------+
|Cilindros|novaCilindros|
+---------+-------------+
|        6|            6|
|        6|            6|
|        0|            6|
|        0|            6|
|        0|            6|
|        6|            6|
|        8|            8|
|        4|            4|
|        4|            4|
|        0|            6|
|        6|            6|
|        8|            8|
|        8|            8|
|        8|            8|
|        8|            8|
|        8|            8|
|        8|            8|
|        4|            4|
|        4|            4|
|        4|            4|
+---------+-------------+
only showing top 20 rows

